# Boto3 SDK를 통해 Agent 생성 및 호출하기

> *이 노트북은 SageMaker Studio*의 **`Data Science 3.0`** kernel과 잘 작동해야 합니다.

## Introduction

이 노트북에서는 `bedrock-agent`와 `bedrock-agent-runtime` boto3 client 사용하는 방법을 설명합니다:
- 에이전트 생성
- API 스키마를 사용하여 액션 그룹 생성(함수 정의 사용과 비교)
- 에이전트를 액션 그룹과 연결하고 에이전트 준비
- 에이전트 alias 생성
- 에이전트 호출

여기서는 Boto3 API를 사용하여 Bedrock's Claude Sonnet를 사용하겠습니다. 

**Note:** *이 노트북은 SageMaker Studio에서 사용하거나 AWS 자격 증명을 설정한 경우 로컬에서 실행할 수 있습니다*.

#### 사전 준비 사항
이 노트북에는 다음 permissions이 필요합니다: 
- Amazon IAM 역할 생성 및 삭제
- AWS Lambda 함수 생성, 업데이트 및 호출 
- Amazon S3 buckets 생성, 업데이트 및 삭제 
- Amazon Bedrock에 액세스 

관리자 역할 없이 이 노트북을 실행하는 경우, 역할에 다음 관리 정책이 포함되어 있는지 확인하세요:
- IAMFullAccess
- AWSLambda_FullAccess
- AmazonS3FullAccess
- AmazonBedrockFullAccess

#### Context
Boto3 SDK를 사용하여 Bedrock용 에이전트를 생성하고 호출하는 방법을 보여드리겠습니다.

#### Use case
이 노트북에서는 에이전트가 보험금 청구 사용 사례의 assistant 역할을 합니다. 에이전트는 보험 직원이 진행 중인 보험금 청구를 확인하고, 특정 보험금 청구에 대한 세부 정보를 식별하고, 보험금 청구에 대한 열린 문서를 가져오고, 보험금 청구자에게 알림을 보낼 수 있도록 도와줍니다.

생성된 에이전트는 다단계 프로세스에서 다음 작업 또는 이러한 작업의 조합을 처리할 수 있습니다:
- Get Open Claims (보험금 청구 열기)
- Get Claim Details (상세 청구 내용)
- Get Claim Outstanding Documents (청구 대표 문서들)
- Send Claim reminder (청구자에게 알림 전송)

<img src="./images/20-architecture-agent-API.png" alt="Create an Agent with Function Definition" style="height: 400px; width:950px;"/>

## Notebook setup

In [7]:
import logging
import boto3
import time
import zipfile
from io import BytesIO
import json
import uuid
import pprint

In [8]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [9]:
# get boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
s3_client = boto3.client('s3')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

[2024-06-07 10:50:20,615] p12271 {credentials.py:1075} INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In [22]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

('us-west-2', '322537213286')

In [23]:
from time import strftime
current_time = strftime("%m%d-%H%M%s")

In [24]:
# Generate random prefix for unique IAM roles, agent name and S3 Bucket and 
# assign variables


suffix = f"{region}-{account_id}"
agent_name = "insurance-claims-agent"
agent_alias_name = "workshop-alias"
bucket_name = f'{agent_name}-{suffix}'
bucket_key = f'{agent_name}-schema.json'
schema_name = 'insurance_claims_agent_openapi_schema.json'
schema_arn = f'arn:aws:s3:::{bucket_name}/{bucket_key}'
bedrock_agent_bedrock_allow_policy_name = f"{agent_name}-allow-{current_time}"
bedrock_agent_s3_allow_policy_name = f"{agent_name}-s3-allow-{current_time}"
lambda_role_name = f'{agent_name}-lambda-role-{current_time}'
agent_role_name = f'AmazonBedrockExecutionRoleForAgents-{current_time}'
lambda_code_path = "lambda_function.py"
lambda_name = f'{agent_name}-{current_time}'
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

### S3 버킷 생성 및 API Schema 업로드하기

Agents에는 s3에 저장된 API Schema가 필요합니다. 파일을 저장할 S3 버킷을 생성하고 새로 생성된 버킷에 파일을 업로드해 보겠습니다.

In [64]:
import json

with open('insurance_claims_agent_openapi_schema.json', 'r') as f:
    json_data = json.load(f)
print(json.dumps(json_data, ensure_ascii=False, indent=2))

{
  "openapi": "3.0.0",
  "info": {
    "title": "Insurance Claims Automation API",
    "version": "1.0.0",
    "description": "APIs for managing insurance claims by pulling list of open claims, identifying outstanding paperwork for each claim, identifying all claim details, and sending reminders to policy holders."
  },
  "paths": {
    "/open-items": {
      "get": {
        "summary": "Gets the list of all open insurance claims",
        "description": "Gets the list of all open insurance claims. Returns all claimIds that are open.",
        "operationId": "getAllOpenClaims",
        "responses": {
          "200": {
            "description": "Gets the list of all open insurance claims for policy holders",
            "content": {
              "application/json": {
                "schema": {
                  "type": "array",
                  "items": {
                    "type": "object",
                    "properties": {
                      "claimId": {
                  

In [25]:
# Create S3 bucket for Open API schema
try:
    if region == "us-east-1":
        s3bucket = s3_client.create_bucket(
            Bucket=bucket_name
        )
    else:
        s3bucket = s3_client.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={ 'LocationConstraint': region } 
        )
except Exception as e:
    print(e)

An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [26]:
# Upload Open API schema to this s3 bucket
s3_client.upload_file(schema_name, bucket_name, bucket_key)

### 액션 그룹을 위한 람다 함수 만들기
이제 agent action group에 필요한 람다 함수를 만들어 보겠습니다. 먼저 람다 IAM 역할과 정책을 만들어야 합니다. 그런 다음 람다 함수를 ZIP 형식으로 패키징하여 함수를 만듭니다.

In [27]:
# Create IAM Role for the Lambda function
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "bedrock:InvokeModel",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_role_name,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_role_name)

iam_client.attach_role_policy(
    RoleName=lambda_role_name,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

{'ResponseMetadata': {'RequestId': 'ddef12b1-ba43-439f-83d0-1eb3f0ff614c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 07 Jun 2024 10:51:21 GMT',
   'x-amzn-requestid': 'ddef12b1-ba43-439f-83d0-1eb3f0ff614c',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

agent의 Action group으로 사용될 람다 함수 코드를 살펴보세요.

In [28]:
!pygmentize lambda_function.py

#!/usr/bin/env python3
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: MIT-0
import json


def get_named_parameter(event, name):
    return next(item for item in event['parameters'] if item['name'] == name)['value']


def get_named_property(event, name):
    return next(
        item for item in
        event['requestBody']['content']['application/json']['properties']
        if item['name'] == name)['value']


def claim_detail(payload):
    claim_id = payload['parameters'][0]['value']
    if claim_id == 'claim-857':
        return {
            "response": {
                "claimId": claim_id,
                "createdDate": "21-Jul-2023",
                "lastActivityDate": "25-Jul-2023",
                "status": "Open",
                "policyType": "Vehicle"
            }
        }
    elif claim_id == 'claim-006':
        return {
            "response": {
                "claimId": claim_id,
                "createdDate": "20-May-2

In [29]:
# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write(lambda_code_path)
z.close()
zip_content = s.getvalue()

# Create Lambda Function
lambda_function = lambda_client.create_function(
    FunctionName=lambda_name,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_function.lambda_handler'
)

### Agent 만들기
에이전트를 생성하겠습니다. 그러기 위해서는 먼저 bedrock 모델 호출과 s3 bucket 액세스를 허용하는 Agent Policy를 만들어야 합니다. 

In [36]:
# Create IAM policies for agent

bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{model_id}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)



EntityAlreadyExistsException: An error occurred (EntityAlreadyExists) when calling the CreatePolicy operation: A policy called insurance-claims-agent-allow-0607-10511717757470 already exists. Duplicate names are not allowed.

다음으로 S3에서 Agent의 OpenAPI 스키마를 가져올 수 있는 Policy를 생성합니다.

In [ ]:
bedrock_agent_s3_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowAgentAccessOpenAPISchema",
            "Effect": "Allow",
            "Action": ["s3:GetObject"],
            "Resource": [
                schema_arn
            ]
        }
    ]
}


bedrock_agent_s3_json = json.dumps(bedrock_agent_s3_allow_policy_statement)
agent_s3_schema_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_s3_allow_policy_name,
    Description=f"Policy to allow invoke Lambda that was provisioned for it.",
    PolicyDocument=bedrock_agent_s3_json
)

2개의 Policy를 추가하여 role을 생성합니다.

In [32]:
# Create IAM Role for the agent and attach IAM policies
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [{
          "Effect": "Allow",
          "Principal": {
            "Service": "bedrock.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
    }]
}

assume_role_policy_document_json = json.dumps(assume_role_policy_document)
agent_role = iam_client.create_role(
    RoleName=agent_role_name,
    AssumeRolePolicyDocument=assume_role_policy_document_json
)

# Pause to make sure role is created
time.sleep(10)
    
iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_bedrock_policy['Policy']['Arn']
)

iam_client.attach_role_policy(
    RoleName=agent_role_name,
    PolicyArn=agent_s3_schema_policy['Policy']['Arn']
)

{'ResponseMetadata': {'RequestId': '4c09a0e2-8edc-4fd8-bc4c-3a226406ceab',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 07 Jun 2024 10:51:33 GMT',
   'x-amzn-requestid': '4c09a0e2-8edc-4fd8-bc4c-3a226406ceab',
   'content-type': 'text/xml',
   'content-length': '212'},
  'RetryAttempts': 0}}

#### Agent 만들기
필요한 IAM 역할이 만들어지면 기반 에이전트 클라이언트를 사용하여 새 에이전트를 만들 수 있습니다. 이를 위해 `create_agent` 함수를 사용합니다. 여기에는 에이전트 이름, 기본 foundation 모델과 instructions이 필요합니다. 에이전트 설명을 제공할 수도 있으며, 생성된 에이전트는 아직 준비되지 않은 상태입니다. 에이전트를 준비한 다음 이를 사용하여 작업을 호출하고 다른 API를 사용하는 데 중점을 두겠습니다.

In [34]:
# Create Agent
agent_instruction = """
너는 보험금 청구와 관련된 다양한 작업을 처리할 수 있는 agent bot입니다. 
세부 정보 조회, 미결 서류 찾기, 알림 전송 등 보험금 청구와 관련된 작업을 처리할 수 있습니다. 
리마인더를 보내도록 명시적으로 요청받은 경우에만 명시적으로 요청받은 경우에만 알림을 보내세요. 
사용자가 기능에 대해 문의하는 경우 자연어로 안내하고, 절대 출력에 function 이름을 포함하지 마세요.
"""

response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description="보험금 청구를 다루는 Agent",
    idleSessionTTLInSeconds=1800,
    foundationModel=model_id,
    instruction=agent_instruction,
)

Looking at the created agent, we can see its status and agent id

In [35]:
response

{'ResponseMetadata': {'RequestId': 'effba8ff-d269-452c-a2ac-74e3fc5d079c',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 07 Jun 2024 10:53:48 GMT',
   'content-type': 'application/json',
   'content-length': '1039',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'effba8ff-d269-452c-a2ac-74e3fc5d079c',
   'x-amz-apigw-id': 'Y_j9iEP2PHcEQyg=',
   'x-amzn-trace-id': 'Root=1-6662e6bc-75f1a1d35d3bfc6a38da6cc3'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-west-2:322537213286:agent/NZDQ4JYPFM',
  'agentId': 'NZDQ4JYPFM',
  'agentName': 'insurance-claims-agent1',
  'agentResourceRoleArn': 'arn:aws:iam::322537213286:role/AmazonBedrockExecutionRoleForAgents-0607-10511717757470',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2024, 6, 7, 10, 53, 48, 896509, tzinfo=tzlocal()),
  'description': '보험금 청구를 다루는 Agent',
  'foundationModel': 'anthropic.claude-3-sonnet-20240229-v1:0',
  'idleSessionTTLInSeconds': 1800,
  'instruction': '\n너는 보험금 청구와 

Let's now store the agent id in a local variable to use it on the next steps

In [37]:
agent_id = response['agent']['agentId']
agent_id

'NZDQ4JYPFM'

### Agent Action Group 만들기
앞서 만든 람다 함수와 API schema 파일을 사용하는 agent action group을 생성해 보겠습니다.
이 기능은 `create_agent_action_group` 함수가 제공합니다. 아직 agent version이나 alias을 만들지 않았으므로 `DRAFT`를 agent version으로 사용하겠습니다. agent에게 action group 기능을 알리기 위해 action group의 기능이 포함된 action group 설명을 제공합니다.

In [38]:
# Pause to make sure agent is created
time.sleep(30)
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName='ClaimManagementActionGroup',
    apiSchema={
        's3': {
            's3BucketName': bucket_name,
            's3ObjectKey': bucket_key
        }
    },
    description='청구 목록, 누락된 서류 확인, 알림 전송을 위한 Actions'
)

In [39]:
agent_action_group_response

{'ResponseMetadata': {'RequestId': '5fcabbef-7993-433b-8411-92c90930ca60',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 07 Jun 2024 12:00:07 GMT',
   'content-type': 'application/json',
   'content-length': '611',
   'connection': 'keep-alive',
   'x-amzn-requestid': '5fcabbef-7993-433b-8411-92c90930ca60',
   'x-amz-apigw-id': 'Y_trBHrxvHcENeQ=',
   'x-amzn-trace-id': 'Root=1-6662f646-45331aa41617d126451c10a3'},
  'RetryAttempts': 0},
 'agentActionGroup': {'actionGroupExecutor': {'lambda': 'arn:aws:lambda:us-west-2:322537213286:function:insurance-claims-agent-0607-10511717757470'},
  'actionGroupId': 'HKVY5SDGUX',
  'actionGroupName': 'ClaimManagementActionGroup',
  'actionGroupState': 'ENABLED',
  'agentId': 'NZDQ4JYPFM',
  'agentVersion': 'DRAFT',
  'apiSchema': {'s3': {'s3BucketName': 'insurance-claims-agent-us-west-2-322537213286',
    's3ObjectKey': 'insurance-claims-agent-schema.json'}},
  'createdAt': datetime.datetime(2024, 6, 7, 12, 0, 7, 95537, tzinfo=tzlocal()),


### Agent가 Action Group Lambda를 호출하도록 허용하기
action group을 사용하기 전에 agent가 action group에 연결된 람다 함수를 호출할 수 있도록 허용해야 합니다. 이는 리소스 기반 정책을 통해 이루어저야 하기에 생성한 람다 함수에 리소스 기반 정책을 추가해 보겠습니다.

In [40]:
# Create allow invoke permission on lambda
response = lambda_client.add_permission(
    FunctionName=lambda_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)

### Agent 준비하기
내부 테스트에 사용할 수 있는 Agent의 DRAFT 버전을 만들어 보겠습니다.

In [41]:
agent_prepare = bedrock_agent_client.prepare_agent(agentId=agent_id)
agent_prepare

{'ResponseMetadata': {'RequestId': 'e17684b2-253a-49d2-84ee-7a2c2ed77e72',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 07 Jun 2024 12:01:32 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e17684b2-253a-49d2-84ee-7a2c2ed77e72',
   'x-amz-apigw-id': 'Y_t4eESmPHcECLQ=',
   'x-amzn-trace-id': 'Root=1-6662f69c-7e07a0ed095958b26da6edbb'},
  'RetryAttempts': 0},
 'agentId': 'NZDQ4JYPFM',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2024, 6, 7, 12, 1, 32, 397347, tzinfo=tzlocal())}

### Agent alias 만들기
이제 agent를 배포하는 데 사용할 수 있는 에이전트의 별칭을 만듭니다.

In [42]:
# Pause to make sure agent is prepared
time.sleep(30)
agent_alias = bedrock_agent_client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name
)

In [43]:
agent_alias

{'ResponseMetadata': {'RequestId': 'bfc8c277-d061-4bc4-bb3c-05827b685519',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Fri, 07 Jun 2024 12:02:44 GMT',
   'content-type': 'application/json',
   'content-length': '340',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'bfc8c277-d061-4bc4-bb3c-05827b685519',
   'x-amz-apigw-id': 'Y_uDsGq9PHcENLA=',
   'x-amzn-trace-id': 'Root=1-6662f6e4-0af6fa7401d5d1d93ef9ec9c'},
  'RetryAttempts': 0},
 'agentAlias': {'agentAliasArn': 'arn:aws:bedrock:us-west-2:322537213286:agent-alias/NZDQ4JYPFM/GARGJJ9UIW',
  'agentAliasId': 'GARGJJ9UIW',
  'agentAliasName': 'workshop-alias',
  'agentAliasStatus': 'CREATING',
  'agentId': 'NZDQ4JYPFM',
  'createdAt': datetime.datetime(2024, 6, 7, 12, 2, 44, 314371, tzinfo=tzlocal()),
  'routingConfiguration': [{}],
  'updatedAt': datetime.datetime(2024, 6, 7, 12, 2, 44, 314371, tzinfo=tzlocal())}}

### Agent 호출
이제 agent를 만들었으니 `bedrock-agent-runtime` 클라이언트를 사용하여 이 agent를 호출하고 몇 가지 작업을 수행해 보겠습니다.

In [44]:
time.sleep(30)
# Extract the agentAliasId from the response
agent_alias_id = agent_alias['agentAlias']['agentAliasId']

In [45]:
## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = False
end_session:bool = False
# Pause to make sure agent alias is ready
# time.sleep(30)

# invoke the agent API
agentResponse = bedrock_agent_runtime_client.invoke_agent(
    inputText="미해결 청구건은 무엇이에요?",
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    endSession= end_session
)

logger.info(pprint.pprint(agentResponse))

[2024-06-07 12:03:30,880] p12271 {574900033.py:18} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/json',
                                      'date': 'Fri, 07 Jun 2024 12:03:30 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': 'f4ee0384-24c5-11ef-83d0-0a86a609e469',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '92c3af44-8344-450c-a09b-724a27e4c0e0'},
                      'HTTPStatusCode': 200,
                      'RequestId': '92c3af44-8344-450c-a09b-724a27e4c0e0',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x7f8e9e171d80>,
 'contentType': 'application/json',
 'sessionId': 'f4ee0384-24c5-11ef-83d0-0a86a609e469'}


In [46]:
%%time
event_stream = agentResponse['completion']
try:
    for event in event_stream:        
        if 'chunk' in event:
            data = event['chunk']['bytes']
            logger.info(f"Final answer ->\n{data.decode('utf8')}")
            agent_answer = data.decode('utf8')
            end_event_received = True
            # End event indicates that the request finished successfully
        elif 'trace' in event:
            logger.info(json.dumps(event['trace'], indent=2))
        else:
            raise Exception("unexpected event.", event)
except Exception as e:
    raise Exception("unexpected event.", e)

[2024-06-07 12:03:52,820] p12271 {<timed exec>:6} INFO - Final answer ->
현재 오픈 상태인 청구건은 총 3건입니다.

청구건 claim-006에 대해서는 AccidentImages 서류가 미해결 상태입니다.

청구건 claim-857에 대해서는 DriverLicense와 VehicleRegistration 서류가 미해결 상태입니다.

청구건 claim-334에 대해서는 모든 서류가 제출되어 미해결 서류가 없습니다.


CPU times: user 3.37 ms, sys: 0 ns, total: 3.37 ms
Wall time: 15.9 s


In [47]:
# And here is the response if you just want to see agent's reply
print(agent_answer)

현재 오픈 상태인 청구건은 총 3건입니다.

청구건 claim-006에 대해서는 AccidentImages 서류가 미해결 상태입니다.

청구건 claim-857에 대해서는 DriverLicense와 VehicleRegistration 서류가 미해결 상태입니다.

청구건 claim-334에 대해서는 모든 서류가 제출되어 미해결 서류가 없습니다.


In [48]:
def simple_agent_invoke(input_text, agent_id, agent_alias_id, session_id=None, enable_trace=False, end_session=False):
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=input_text,
        agentId=agent_id,
        agentAliasId=agent_alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session
    )
    
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:        
            if 'chunk' in event:
                data = event['chunk']['bytes']
                logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                # End event indicates that the request finished successfully
            elif 'trace' in event:
                logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)

In [49]:
simple_agent_invoke("claim-857에 대해 알려줘", agent_id, agent_alias_id, session_id)

[2024-06-07 12:27:03,844] p12271 {3357547408.py:16} INFO - Final answer ->
청구건 claim-857에 대한 세부 정보는 다음과 같습니다:

- 청구건 ID: claim-857
- 생성일자: 21-Jul-2023  
- 마지막 활동일: 25-Jul-2023
- 현재 상태: Open
- 보험 종류: Vehicle

이 청구건에 대해 아직 제출되지 않은 서류는 DriverLicense와 VehicleRegistration입니다.


In [50]:
simple_agent_invoke("서류가 누락된 모든 미결 청구에 대한 알림을 보내줘", agent_id, agent_alias_id, session_id)

[2024-06-07 12:28:16,964] p12271 {3357547408.py:16} INFO - Final answer ->
모든 미해결 서류가 있는 오픈 청구건에 대해 알림을 성공적으로 보냈습니다.

청구건 claim-006에 대해서는 AccidentImages 서류 제출을 요청하는 알림을 보냈습니다.

청구건 claim-857에 대해서는 DriverLicense와 VehicleRegistration 서류 제출을 요청하는 알림을 보냈습니다.

청구건 claim-334에는 미해결 서류가 없어 알림을 보내지 않았습니다.


### Clean up(선택 사항)
다음 단계는 선택 사항이며 에이전트를 삭제하는 방법을 설명합니다. 에이전트를 삭제하려면 다음과 같이 해야 합니다:
1. action group을 비활성화로 업데이트
2. agent action group 삭제
3. agent alias 삭제
4. agent 삭제
5. 람다 함수 삭제
6. 생성된 S3 버킷 비우기
7. 생성된 S3 버킷 삭제

In [51]:
 # This is not needed, you can delete agent successfully after deleting alias only
# Additionaly, you need to disable it first

action_group_id = agent_action_group_response['agentActionGroup']['actionGroupId']
action_group_name = agent_action_group_response['agentActionGroup']['actionGroupName']

response = bedrock_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id,
    actionGroupName=action_group_name,
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    apiSchema={
        's3': {
            's3BucketName': bucket_name,
            's3ObjectKey': bucket_key
        }
    },
    actionGroupState='DISABLED',
)

action_group_deletion = bedrock_agent_client.delete_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id
)

In [52]:
agent_alias_deletion = bedrock_agent_client.delete_agent_alias(
    agentId=agent_id,
    agentAliasId=agent_alias['agentAlias']['agentAliasId']
)

In [53]:
agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

In [54]:
# Delete Lambda function
lambda_client.delete_function(
    FunctionName=lambda_name
)

{'ResponseMetadata': {'RequestId': 'b4d9bc0c-8af3-4f30-bdcc-7954298a373e',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'date': 'Fri, 07 Jun 2024 12:31:21 GMT',
   'content-type': 'application/json',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'b4d9bc0c-8af3-4f30-bdcc-7954298a373e'},
  'RetryAttempts': 1}}

In [55]:
# Empty and delete S3 Bucket

objects = s3_client.list_objects(Bucket=bucket_name)  
if 'Contents' in objects:
    for obj in objects['Contents']:
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key']) 
s3_client.delete_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'HVTT1NZW87HK83TC',
  'HostId': '0YGwvKY/clkN7k7l354urQ7GeCjdEqpXMuBZlV4K9hX8r2A6oQ/1N0sGjAjLHJrnF0zfUgxBuziffiAQw6BPVQ==',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': '0YGwvKY/clkN7k7l354urQ7GeCjdEqpXMuBZlV4K9hX8r2A6oQ/1N0sGjAjLHJrnF0zfUgxBuziffiAQw6BPVQ==',
   'x-amz-request-id': 'HVTT1NZW87HK83TC',
   'date': 'Fri, 07 Jun 2024 12:31:22 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [56]:
# Delete IAM Roles and policies

for policy in [bedrock_agent_bedrock_allow_policy_name, bedrock_agent_s3_allow_policy_name]:
    iam_client.detach_role_policy(RoleName=agent_role_name, PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}')
    
iam_client.detach_role_policy(RoleName=lambda_role_name, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')

for role_name in [agent_role_name, lambda_role_name]:
    iam_client.delete_role(
        RoleName=role_name
    )

for policy in [agent_bedrock_policy, agent_s3_schema_policy]:
    iam_client.delete_policy(
        PolicyArn=policy['Policy']['Arn']
)

## Conclusion
We have now experimented with using `boto3` SDK to create, invoke and delete an agent.

### Take aways
- Adapt this notebook to create new agents for your application

## Thank You